In [4]:
import tensorflow as tf
import keras
from transformers import GPT2Tokenizer
import re
from sklearn.metrics import roc_auc_score
from sklearn.metrics import roc_curve, auc
import numpy as np 
import matplotlib.pyplot as plt
import pandas as pd

In [13]:
model_dir = "../../dem_rep/dem_rep_model"
democrat_file = "../../newtweets/democrat.csv"
republican_file = "../../newtweets/republican.csv"
model_name = "gpt2"

In [6]:
batch_size = 8

In [7]:
tokenizer = GPT2Tokenizer.from_pretrained(model_name)
tokenizer.pad_token = tokenizer.eos_token

In [8]:
new_model = tf.keras.models.load_model("../../dem_rep/dem_rep_model")

In [9]:
def predict_news_class(probabilities, threshold=0.5):
    republican_probability, democrat_probability = probabilities[0]
    if (republican_probability - democrat_probability)**2 > 0.3:
        if republican_probability > threshold:
            return "republican", republican_probability, democrat_probability, ((republican_probability - democrat_probability)**2)
        else:
            return "democrat", republican_probability, democrat_probability, ((republican_probability - democrat_probability)**2)
    else:
        return "neutral", republican_probability, democrat_probability, ((republican_probability - democrat_probability)**2)

In [10]:
def remove_emojis_and_links(text):
    text = re.sub(r'[^\x00-\x7F]+', '', text)
    text = re.sub(r'http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\\(\\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+', '', text)
    text = re.sub(r'@\w+', '', text)
    text = re.sub(r'[^a-zA-Z0-9\s]|RT', '', text)
    text = re.sub(r'\s+', ' ', text)
    text = text.replace("\n", "")
    return text.strip()

In [11]:
inputs = tokenizer(remove_emojis_and_links(input()), return_tensors='tf', max_length=512, truncation=True, padding='max_length')
output = new_model(inputs)
print(predict_news_class(output), sep = "\n")

('republican', <tf.Tensor: shape=(), dtype=float32, numpy=0.95282745>, <tf.Tensor: shape=(), dtype=float32, numpy=0.047172535>, <tf.Tensor: shape=(), dtype=float32, numpy=0.8202108>)
